## Employment Claims and Housing Prices

In [1]:
# Load up necessary packagaes
import pandas as pd
import numpy as np

from scipy.stats import ttest_ind
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor
#from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

## Housing Price Data
Load Housing Data from file and pre-process it

In [2]:
raw_zHousing_df = pd.read_csv('State_Zhvi_AllHomes.csv')

In [3]:
State_HP_df = raw_zHousing_df.drop(['RegionID','RegionType','StateName', 'SizeRank'], axis = 1)
State_HP_df = State_HP_df.set_index(['RegionName'])
State_HP_df = State_HP_df.T
State_HP_df['Date'] = pd.to_datetime(State_HP_df.index)
#zHousing_df['month'] = zHousing_df['Date'].dt.month
State_HP_df['year'] = State_HP_df['Date'].dt.year
State_HP_df['week'] = State_HP_df['Date'].dt.week
State_HP_df = State_HP_df.set_index(['year','week'])

State_HP_df = State_HP_df.drop(['Date'], axis=1)
#State_HP_df.head()

Load data for individual counties

In [4]:
raw_cityHousing_df = pd.read_csv('City_Zhvi_AllHomes.csv')

In [97]:
countyHP_df = raw_cityHousing_df.drop(['RegionName','RegionType', 'SizeRank','Metro','State','CountyName'], axis = 1)

countyHP_df = countyHP_df.drop(['StateName'], axis=1)
countyHP_df = countyHP_df.set_index(['RegionID'])
countyHP_df = countyHP_df.T
countyHP_df['Date'] = pd.to_datetime(countyHP_df.index)
countyHP_df['year'] = countyHP_df['Date'].dt.year
countyHP_df['week'] = countyHP_df['Date'].dt.week
countyHP_df = countyHP_df.set_index(['year','week'])

countyHP_df = countyHP_df.drop(['Date'], axis=1)
countyHP_df.fillna(method='ffill', inplace=True)
countyHP_df.dropna(inplace=True, axis=1)

In [147]:
#countyHP_df.head()

## Insurance Claims Data

Pre-process the insurance claims data

In [7]:
raw_Uinsurance_df = pd.read_csv('r539cy-master.xls.csv')

In [8]:
Uinsurance_df = raw_Uinsurance_df
Uinsurance_df['Date'] = pd.to_datetime(raw_Uinsurance_df['Filed week ended'])
#Uinsurance_df['month'] = Uinsurance_df['Date'].dt.month
Uinsurance_df['year'] = Uinsurance_df['Date'].dt.year
Uinsurance_df['week'] = Uinsurance_df['Date'].dt.week

Uinsurance_df = Uinsurance_df.drop(['Date'], axis=1)
# drop the records before 1996, to match the data  that is available from Zillow
Uinsurance_df = Uinsurance_df[Uinsurance_df['year'] > 1995]

Uinsurance_df['Initial Claims'] = Uinsurance_df['Initial Claims'].str.replace(',', '').astype(int)
Uinsurance_df['Continued Claims'] = Uinsurance_df['Continued Claims'].str.replace(',', '').astype(int)
Uinsurance_df['Covered Employment'] = Uinsurance_df['Covered Employment'].str.replace(',', '').astype(int)

In [146]:
#Uinsurance_df.head()

Transform unemployment insurance data into format that can be merged with the housing data

In [10]:
ui_initclaims_df = pd.pivot_table(Uinsurance_df, values='Initial Claims', index=['year', 'week'], columns=['State'], aggfunc=np.sum)
ui_contclaims_df = pd.pivot_table(Uinsurance_df, values='Continued Claims', index=['year', 'week'], columns=['State'], aggfunc=np.sum)
ui_rate_df = pd.pivot_table(Uinsurance_df, values='Insured Unemployment Rate', index=['year', 'week'], columns=['State'], aggfunc=np.sum)
ui_covEmp_df = pd.pivot_table(Uinsurance_df, values='Covered Employment', index=['year', 'week'], columns=['State'], aggfunc=np.sum)

Create look-up table to match the county to the appropriate state

In [139]:
state_abbrvMap = raw_cityHousing_df.dropna(axis=0)
state_abbrvMap = state_abbrvMap[['RegionID', 'RegionName','State', 'CountyName']]
Full_stateName = raw_zHousing_df[['RegionName','StateName']]
Full_stateName.columns = ['StateName','Abbreviation']
County_State_df = pd.merge(state_abbrvMap, Full_stateName, how='left', left_on='State', right_on='Abbreviation')

In [145]:
#County_State_df.head()

### Combine Data
Combine UI and Housing Price data for each State into DataFrame and Engineer training featues

In [132]:
def get_County_df (countyID):    
        
    # get full state name from state housing price datafile
    tmp_lookup_row = County_State_df[County_State_df['RegionID'] == countyID]
    stateName = tmp_lookup_row['StateName'].to_numpy()[0]
    
    tmp_CntyHousingPrc_df = countyHP_df[countyID]
    
    df = pd.DataFrame([])
    df['Initial Claims'] = ui_initclaims_df[stateName]
    df['Continued Claims'] = ui_contclaims_df[stateName]
    df['Covered Employment'] = ui_covEmp_df[stateName]
    df['Insured Unemployment Rate'] = ui_rate_df[stateName]
    
    df = pd.merge(df, tmp_CntyHousingPrc_df, how='left', left_index=True, right_index=True)    
    df[countyID].fillna(method='ffill', inplace=True)
    df.fillna(0, inplace=True)
    
    df['housingTarget'] = round(df[countyID].astype(int) / 1000, 1)
        
    if countyID in df.columns:
        df.sort_index(ascending=True, inplace=True)
        df = df.reset_index()

        df.fillna(0, inplace=True)        
        df['PrevHousePrc'] = df['housingTarget'].rolling(2, min_periods=1).sum() - df['housingTarget']
        df['MvngAvgPrc15'] = round(df['PrevHousePrc'].rolling(10).mean())
        df['MvngAvgPrc30'] = round(df['PrevHousePrc'].rolling(15).mean())    

        df = df.drop([countyID], axis=1)
        
    df.fillna(method='ffill', inplace=True)
    df.fillna(0, inplace=True)
    
    df = df.iloc[1:]  
    test_df = df.tail(1)
    df.drop(df.tail(1).index, inplace=True)
    
    
    return df, test_df

In [148]:
#df, test = get_County_df (8097)
#test

,year,week,Initial Claims,Continued Claims,Covered Employment,Insured Unemployment Rate,housingTarget,PrevHousePrc,MvngAvgPrc15,MvngAvgPrc30
1264,2020,15,222207,749011,4305711,17.4,399.4,399.4,400.0,400.0


#### Correlation Test Function
Test how strongly the values are co-related

In [50]:
def get_State_Prop (df):
    
    fin_r = stats.ttest_ind(df['Initial Claims'], df['housingTarget'], equal_var=False)
    return fin_r[0], fin_r[1], len(df['housingTarget'].unique())

In [51]:
df, test_df = get_County_df ('Ann Arbor')
get_State_Prop (df)

(32.96657568902683, 1.9157875909124523e-172, 261)

### Train GBT Model

In [150]:
def train_StateModels():

    #scounties = County_State_df[County_State_df['StateName'] == 'Michigan']
    #counties = counties['RegionID'].to_numpy()
    counties = ['Ann Arbor','Ypsilanti']
    
    for county in counties:

        df, f_test = get_County_df(county)
        
        # features
        X = df[['Initial Claims','Continued Claims','Insured Unemployment Rate','Covered Employment','PrevHousePrc']]
        # target
        y = df[['housingTarget']]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

        clf = DecisionTreeRegressor(random_state = 42, max_depth = 4)
        clf.fit(X_train, y_train)

        y_predicted = clf.predict(f_test[['Initial Claims','Continued Claims','Insured Unemployment Rate','Covered Employment','PrevHousePrc']]) 
        
        state_Results.append([county, clf.score(X_train, y_train), clf.score(X_test, y_test), y_predicted[0], f_test['housingTarget'].to_numpy()[0]])
        
    return state_Results

#train_StateModels()

#### Feature Assessment for GBDT
Call the grid search to identify which parameters work be

## MLP Regressor

In [149]:
# MLP CLassifier Model
def train_StateMLPMdls():
    
    counties = County_State_df[County_State_df['StateName'] == 'Michigan']
    counties = counties['RegionID'].to_numpy()
    #counties = ['Ann Arbor','Ypsilanti']
    
    for countyID in counties:
        
        df, f_test = get_County_df(countyID)

        X = df.drop(['housingTarget'], axis=1)
        y = df[['housingTarget']]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        scaler = MinMaxScaler()

        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_ftest_scaled = scaler.transform(X.iloc[-1:])
                               
        mlp_clf = MLPRegressor(hidden_layer_sizes = [100, 100], solver='lbfgs', alpha=0.15, random_state=42).fit(X_train_scaled, y_train)                
        y_predicted = mlp_clf.predict(X_ftest_scaled) 
        
        state_Results.append([countyID, mlp_clf.score(X_train_scaled, y_train), mlp_clf.score(X_test_scaled, y_test), y_predicted[0], f_test['housingTarget'].to_numpy()[0]])
        
    return state_Results


In [144]:
def save_results(model):

    if 'mlp' in model:
        State_results_df = pd.DataFrame(train_StateMLPMdls())
        #State_results_df.columns = ['State','Train Score', 'Test Score', 'Statistic', 'Pvalue', 'Unique Targets','Predicted', 'Actual']
        State_results_df.to_csv('countyResults_mlp.csv', index=False)
    else: 
        State_results_df = pd.DataFrame(train_StateModels())
        #State_results_df.sort_values(by=['Test Score'], ascending=False, inplace=True)  
        #State_results_df.columns = ['State','Train Score', 'Test Score', 'Statistic', 'Pvalue', 'Unique Targets','Predicted', 'Actual']
        State_results_df.to_csv('countyResults_gbc.csv', index=False)
    
    return State_results_df[2].to_numpy().mean()
    
save_results('mlp')

0.9906253451245424

# Sources

GDP
#https://apps.bea.gov/regional/Downloadzip.cfm
#https://apps.bea.gov/itable/iTable.cfm?ReqID=70&step=1

Unemployment
#https://oui.doleta.gov/unemploy/claims_arch.asp